In [1]:
import speech_recognition as sr
import os
import sys
import re
import webbrowser
import smtplib
import requests
import subprocess
from pyowm import OWM
import urllib.request
import pyttsx3
import json
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import wikipedia
import random
from time import strftime
from gtts import gTTS
import time
import datetime
import wolframalpha 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pafy
import googletrans
from googlesearch import search
from googletrans import Translator
from pygame import mixer

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Helper Functions

In [2]:
#Administer
def M(audio):
    print(audio)
    engine=pyttsx3.init()
    voicid="com.apple.speech.synthesis.voice.lee.premium"
    engine.setProperty('voice', voicid)
    engine.say(audio)
    engine.runAndWait()


def myCommand():
    "listens for commands"
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print('Say something...')
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
    try:
        command = r.recognize_google(audio).lower()
        print('You said: ' + command + '\n')
    #loop back to continue to listen for commands if unrecognizable speech is received
    except sr.UnknownValueError:
        print('....')
        command = myCommand();
    return command

#Translator
def translate(command:str,text:str):
    'Translates the text to the command'
    glossary=googletrans.LANGUAGES
    glos=list(glossary)
    for p in glos:
        
        if googletrans.LANGUAGES[p]==command:
            translator=Translator()
            translated = translator.translate(text, src='en', dest=p)
            t = gTTS(text=translated.text, lang=p)
            t.save(p+'.mp4')
            mixer.init()
            mixer.music.load(p+".mp4")
            mixer.music.play()
        else:
            pass
    return 

#SEARCH MODULE
def search_web(command): 
    """Returns results on google for the given command"""

    chromedriver = '/Users/Kantemirovs/Desktop/chromedriver'
    driver = webdriver.Chrome(executable_path=chromedriver) #chromedriver version: 79.0.3945.36
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(1) 
    driver.maximize_window() 
    M("Searching")
    driver.get("https://www.google.com/")
    search_bar = driver.find_element_by_name('q')
    button = driver.find_element_by_name('btnK')
    if 'google' in command: 
        command = command.replace('google',"")
        search_bar.send_keys(command)
        button.click()
    elif 'search' in command: 
        command = command.replace('search',"")
        search_bar.send_keys(command)
        button.click()
  
    else: 
        pass
        
    return

def searcher(st:str)-> list:
    """Searches the command and returns a list of resulting websites"""
    
    try: 
        from googlesearch import search
        
    except ImportError:  
        print("No module named 'google' found") 

    results=[]
    for j in search(st, tld="co.in", num=10, stop=3, pause=3): 
        results.append(j)
            
    return results

#Music
def play(command:str):
    """Takes the command and plays it on youtube"""
    
    chromedriver = '/Users/Kantemirovs/Desktop/chromedriver'
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = webdriver.Chrome(chromedriver)
    driver.implicitly_wait(1) 
    driver.maximize_window()  

    M("Opening in youtube") 
    indx = command.replace('play',"")  
    driver.get("https://www.youtube.com/results?search_query=" + indx)
    wait = WebDriverWait(driver, 3)
    presence = EC.presence_of_element_located
    visible = EC.visibility_of_element_located


        # play the video
    wait.until(visible((By.ID, "video-title")))
    driver.find_element_by_id("video-title").click()
    return
#Everything Else
def srchr(query:str):
    """Takes the command and performs google search"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessary.
    chromedriver = '/Users/Kantemirovs/Desktop/chromedriver'
    driver = webdriver.Chrome(chromedriver,options=options) 
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(1) 
    driver.maximize_window() 
    #M("Searching")
    driver.get("https://www.google.com/")
    search_bar = driver.find_element_by_name('q')
    #button = driver.find_element_by_name('btnK')
    search_bar.send_keys(query)
    search_bar.send_keys(Keys.RETURN);
    labels = driver.find_elements_by_css_selector(".e24Kjd") 
    panels=driver.find_elements_by_css_selector(".bNg8Rb+ span")
    tbl=driver.find_elements_by_css_selector('.iKJnec')
    liss=driver.find_elements_by_css_selector('.bOYhNc')
    pr=driver.find_elements_by_css_selector('.NprOob')
    temp=driver.find_elements_by_css_selector('#wob_tm')
    dat=driver.find_elements_by_css_selector('.Z0LcW')
    res=driver.find_elements_by_css_selector('.dbg0pd span')
    timr=driver.find_elements_by_css_selector('.dDoNo')
    defin=driver.find_elements_by_css_selector('.QIclbb span')
   # search_result_list = list(search(query, tld="com", num=10, stop=3, pause=1))
    if "price" in query or "closing price" in query:
        for p in pr:
            m=M(p.text+'$')
            driver.close()
            return m
   
    elif len(liss)>0:
        for l in liss:
            m=M(l.text)
        
        driver.close()
        return m
    
    elif len(temp)>0:
        for te in temp:
            m=M("It's"+ te.text +"Celsius")
            driver.close()
            return m
    elif len(timr)>0:
        for tim in timr:
            m=M("It's"+ tim.text)
            
        driver.close()
        return m
    elif len(res)>0:
        for r in res:
            m=M(r.text)
            
        driver.close()
        return m
    elif len(defin)>0:
        for de in defin:
            m=M(de.text)
            driver.close()
            return m
    elif len(dat)>0:
        for d in dat:
            m=M(d.text)
            
        driver.close()
        return m
    elif len(tbl)>0:
        for t in tbl:
            m=M(t.text)
            
        driver.close()
        return m
    elif len(labels)<len(panels) and len(labels)>0:
        for label in labels:
            m=M(label.text)
            
        driver.close()
        return m
        
    else:
        if len(panels)>0:
            for panel in panels:
                m=M(panel.text)
        
            driver.close()
            return m

        elif len(panels)== 0:
            M('I found these on '+  query)
            m=searcher(query)
            
            return print(m)
            
    return m



In [ ]:
def assistant(command):
    "if statements for executing commands"

    #open subreddit Reddit
    if 'nothing' in command or 'abort' in command or 'stop' in command:
        M('okay')
       
    if 'shutdown' in command:
        M('Okay, bye Sir.')
        os.system('shutdown -s')
    #open website
    elif 'open' in command:
        reg_ex = re.search('open (.+)', command)
        if reg_ex:
            domain = reg_ex.group(1)
            print(domain)
            url = 'https://www.' + domain+'.com'
            webbrowser.open(url)
            M(command +' has been opened for you Sir.')
        else:
            pass

    #greetings
    elif 'hello' in command:
        day_time = int(strftime('%H'))
        if day_time < 12:
            M('Hello Sir. Good morning')
        elif 12 <= day_time < 18:
            M('Hello Sir. Good afternoon')
        else:
            M('Hello Sir. Good evening')
    elif "what\'s up" in command or 'how are you' in command:
        stMsgs = ['Just doing my thing!', 'I am fine!', 'Nice!', 'I am nice and full of energy']
        M(random.choice(stMsgs))
                
    #google search
    elif 'search'  in command or "google" in command: 
        search_web(command)  

    #top stories from google news
    elif 'news for today' in command:

            news_url="https://news.google.com/news/rss"
            Client=urlopen(news_url)
            xml_page=Client.read()
            Client.close()
            soup_page=soup(xml_page,"xml")
            news_list=soup_page.findAll("item")
            for news in news_list[:5]:
                time.sleep(7)
                M(news.title.text)
                M.r
#title.text.encode('utf-8')news.title.text)
    elif 'email' in command:
        M('Who is the recipient?')
        recipient = myCommand()
        M('What should I say to him?')
        content = myCommand()
        mail = smtplib.SMTP('kandemircj.gmail.com', 587)
        mail.ehlo()
        mail.starttls()
        mail.login('kandemircj@gmail.com', '****')
        mail.sendmail('kandemircj@gmail.com', recipient, content)
        mail.close()
        M('Email has been sent successfuly. You can check your inbox.')
        


    #launch any application
    elif 'launch' in command:
        reg_ex = re.search('launch (.*)', command)
        if reg_ex:
            appname = reg_ex.group(1)
            appname1 = appname+".app"
            subprocess.Popen(["open", "-n", "/Applications/" + appname1], stdout=subprocess.PIPE)
        M('I have launched the desired application')

    elif 'set a reminder' in command or 'reminder' in command:
            M("What for?")
            
            text = myCommand()
            
            M("For when?")
            local_time = myCommand()
            
            if 'minutes' in local_time:
                local_time=local_time.replace('minutes',"")
                local_time = float(local_time) *60
                M('All set')
                time.sleep(local_time)
                M('Sir, you have'+text)
                
            elif 'hours' in local_time:
                local_time=local_time.replace('hours',"")
                local_time = float(local_time)*60*60
                M('All set')
                time.sleep(local_time)
                M('Sir, you have'+text)
                
            elif 'a.m.' in local_time or 'p.m.' in local_time or 'day' in local_time:
                local_time=local_time.replace('a.m.','AM')
                local_time=local_time.replace('p.m.','PM')
                M('All set')
                if tym(local_time) is True:
                            M('Sir, you have'+text)
                elif day(local_time) is True:    
                            M('Sir, you have'+text)
                        

            else:
                M('Sir, be clear')
    elif "calculate" in command: 
        app_id = "G77HGG-A6KWYW3U8W" 
        client = wolframalpha.Client(app_id) 
  
        indx = command.lower().split().index('calculate') 
        query = command.split()[indx + 1:] 
        res = client.query(' '.join(query)) 
        answer = next(res.results).text 
        M("The answer is " + answer) 
        
    elif 'translate' in command:
        M("To what?")
        command=myCommand()
        M("What do you want to translate?")
        text=myCommand()
        
        translate(command,text)

    #play youtube song
    elif 'play' in command:
        play(command)
    #joke
    elif 'joke' in command:
        res = requests.get(
                'https://icanhazdadjoke.com/',
                headers={"Accept":"application/json"}
                )
        if res.status_code == requests.codes.ok:
            M(str(res.json()['joke']))
        else:
            M('oops!I ran out of jokes')
                    
    #ask me anything
    elif 'tell me about' in command:
        M('Searching Wikipedia...')
        command = command.replace("tell me about", "")
        results = wikipedia.summary(command, sentences=2)
        M("According to Wikipedia")
        print(results)
        M(results)
    #todays date
    elif 'today' and 'date' in command:
        M(getDate())
   #finds the location
    elif "where is" in command or "where's" in command:
          
        chromedriver = '/Users/Kantemirovs/Desktop/chromedriver'
        driver = webdriver.Chrome(chromedriver)
        command = command.split(" ")
        location_url = "https://www.google.com/maps/place/" + str(command[2])
        M("I will show you where " + command[2] + " is.")
        driver.get(location_url)
    
    else:
        srchr(command)
        

#loop to continue executing multiple commands

while True:
    command=myCommand()
    if 'hey' in command and 'jay' in command:
        M('What can I do for you?')
        assistant(myCommand())
        
    else:
        command